## Notebook to check VTK functionality

In [1]:
import vtk
import vtk_functions
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import os


### Load CSD data and compute mesh

In [2]:
file_list = os.listdir('/home/ntolley/Jones_Lab/lfp_reeb_local/data/gbarEvPyrAmpa_sweep/points')

file_name = '/home/ntolley/Jones_Lab/lfp_reeb_local/data/gbarEvPyrAmpa_sweep/points/gbarEvPyrAmpa_sweep10.csv'

grid = vtk_functions.csd_to_mesh(file_name)
decimated = vtk_functions.decimate_mesh(grid, 0.99)

reeb_graph = vtk.vtkReebGraph()
err = reeb_graph.Build(decimated, decimated.GetPointData().GetScalars())

vertex_list, edge_list = vtk_functions.reeb_to_skeleton(reeb_graph,decimated)

G = nx.Graph()
G.add_edges_from(edge_list)
nx.is_tree(G)

### Visualize Graph

In [5]:
%matplotlib qt

surface_points = np.array(pd.read_csv(file_name)) 

node_points = np.array(vertex_list)
node_connectivity = np.array(edge_list)

num_pairs = node_connectivity.shape[0]
fig = plt.figure(figsize = (8,6))
ax = plt.axes(projection='3d')

for pair in range(num_pairs):
    pairID = node_connectivity[pair]
    xdata, ydata, zdata = node_points[pairID, 0], node_points[pairID, 1], node_points[pairID, 2]

    ax.plot(xdata,ydata,zdata, 'k', linewidth=0.2)

step_size = 50
ax.plot_trisurf(surface_points[::step_size,0],surface_points[::step_size,1],surface_points[::step_size,2],cmap='viridis',edgecolor='none', alpha=0.8)
plt.show()